## Model B: 
* 2 Convolutional Layers
    * Same Padding (same output size)
* 2 **Average** Pooling Layers
* 1 Fully Connected Layer

### Steps
* STEP 1: Load dataset
* STEP 2: Make dataset iterable
* STEP 3: Create model class
* STEP 4: Instantiate model class
* STEP 5: Instantiate loss class
* STEP 6: Instantiate optimizer class
* Step 7: Train Model! 

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
'''
Step 1: Load the dataset
'''

train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

'''
Step 2: Make dataset iterable
'''

batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))  #Return as type INT

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

'''
Step 3: Create Model Class (relu)
'''

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Average Pooling 1
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Average Pooling 2
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        
        # Fully Connected Layer
        self.fc1 = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Avg Pooling 1
        out = self.avgpool1(out)
        
        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Avg Pooling 2
        out = self.avgpool2(out)
        
        # Resize for the linear function in the readout layer
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # Desired out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)  #I might need to look into this to see what is going on under the hood
        
        # Linear Function (readout) Layer
        out = self.fc1(out)
        
        return out
'''
Step 4: Instantiate Model Class
'''

model = CNNModel()

'''
Step 5: Instantiate Loss Class
'''

criterion = nn.CrossEntropyLoss()

'''
Step 6: Instantiate Optimizer Class
'''

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
Step 7: Train the Model
'''

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
    
        images = Variable(images) #No need to resize, because the image is already in a form that the CNN can use.
        labels = Variable(labels)
        
        #Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs / logits
        outputs = model(images)
        
        #Calculate Loss: softmax --> Cross Entropy Loss
        loss = criterion(outputs, labels)
        
        #Get gradients w.r.t. parameters
        loss.backward()
        
        #Update parameters
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0 
            total = 0
            #Iterate through the test dataset
            for images, labels in test_loader:
                # Load images to Torch Variable
                images = Variable(images)
                
                # Forward pass only to get outputs/logits
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
                    
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Iteration: 500. Loss: 0.6557086706161499. Accuracy: 87.44
Iteration: 1000. Loss: 0.2707046866416931. Accuracy: 89.35
Iteration: 1500. Loss: 0.4009123146533966. Accuracy: 90.15
Iteration: 2000. Loss: 0.34728553891181946. Accuracy: 91.1
Iteration: 2500. Loss: 0.6111747622489929. Accuracy: 91.89
Iteration: 3000. Loss: 0.24486079812049866. Accuracy: 93.37


## Avg Pooling Test Accuracy < Max Pooling Test Accuracy! 

## Model C:
* 2 Convolutional Layers
    * **Valid** Padding (smaller output size)
* 2 **Max** Pooling Layers
* 1 Fully Connected Layer

**Valid Padding**
* $O=\frac{W-K+0}{S}+1$
    * $O$: output height/length
    * $W$: input height/length
    * $K$: filter size / kernel size: **5**
    * $P$: padding: **valid padding**
        * $P=\frac{K-5}2=\frac{5-5}2=2$
    * $S$: stride: **=1**
**Output size for Max Pooling**
* $O=\frac WK$
    * $W$: Input height/width
    * $K$: **filter size = 2** (downsamples to half input size, i.e. 2 in the denominator)

In [3]:
# I can start with step 3, to save time in recreating the model and code to run the model. 

'''
Step 3: Create Model Class (relu)
'''

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max Pooling 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max Pooling 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully Connected Layer
        self.fc1 = nn.Linear(32 * 4 * 4, 10)  # Instructor didn't mention this difference when using valid padding (size of array going to fc layer)
        
    def forward(self, x):
        
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max Pooling 1
        out = self.maxpool1(out)
        
        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max Pooling 2
        out = self.maxpool2(out)
        
        # Resize for the linear function in the readout layer
        # Original size: (100, 32, 4, 4)  Instructor didn't mention this difference when using valid padding
        # out.size(0): 100
        # Desired out size: (100, 32*4*4)  
        out = out.view(out.size(0), -1)  #I might need to look into this to see what is going on under the hood
        
        # Linear Function (readout) Layer
        out = self.fc1(out)
        
        return out
'''
Step 4: Instantiate Model Class
'''

model = CNNModel()

'''
Step 5: Instantiate Loss Class
'''

criterion = nn.CrossEntropyLoss()

'''
Step 6: Instantiate Optimizer Class
'''

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
Step 7: Train the Model
'''

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
    
        images = Variable(images) #No need to resize, because the image is already in a form that the CNN can use.
        labels = Variable(labels)
        
        #Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs / logits
        outputs = model(images)
        
        #Calculate Loss: softmax --> Cross Entropy Loss
        loss = criterion(outputs, labels)
        
        #Get gradients w.r.t. parameters
        loss.backward()
        
        #Update parameters
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0 
            total = 0
            #Iterate through the test dataset
            for images, labels in test_loader:
                # Load images to Torch Variable
                images = Variable(images)
                
                # Forward pass only to get outputs/logits
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
                    
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Iteration: 500. Loss: 0.45052123069763184. Accuracy: 88.78
Iteration: 1000. Loss: 0.42741620540618896. Accuracy: 92.22
Iteration: 1500. Loss: 0.22582270205020905. Accuracy: 94.67
Iteration: 2000. Loss: 0.23617702722549438. Accuracy: 95.66
Iteration: 2500. Loss: 0.10575290769338608. Accuracy: 96.34
Iteration: 3000. Loss: 0.09704292565584183. Accuracy: 96.59


### Summary of Results 

**Model A** | **Model B** | **Model C**
--- | --- | ---
Max Pooling | Average Pooling | Max Pooling
Same Padding | Same Padding | Valid Padding
97.07% | 93.33% | 96.59%

| **All Models** |
|---|
|INPUT$\to$CONV$\to$POOL$\to$CONV$\to$POOL$\to$FC|
|Convolution Kernel Size = 5$\times$5|
|Convolution Kernel Stride = 1|
|Pooling Kernel Size = 2$\times$2|

### Expanding a Model's Capacity

* 3 ways to expand a convolutional neural network
    * More convolutional layers
    * Less aggressive downsampling
        * Smaller kernel size for pooling (gradually downsampling)
    * More fully connected layers
* Cons
    * Need a larger dataset
        * Curse of dimensionality
    * Does not necessarily mean higher accuracy